In [1]:
import findspark
findspark.init()
from pyspark import SparkContext

sc = SparkContext("local","quiz")

In [2]:
import numpy as np
import math

In [17]:
data = sc.textFile('./cardio_train.csv').map(lambda x : (np.array([float(y)/10000 for y in x.split(';')[1:12]]),int(x.split(';')[-1]))).cache()

temp = data.collect()
train_set = sc.parallelize(temp[:42000]).cache()
test_set = sc.parallelize(temp[42000:]).cache()

w = np.random.permutation(11)
temp = [float(a) for a in w]

w = np.array(temp)


learning_rate = 0.0001
for _ in range(100):
    gradient = train_set.map(lambda p:(p[1] - 1 / (1 + math.exp(-np.dot(w,p[0])))) * p[0]).reduce(lambda a, b: a + b)
    w += learning_rate * np.array(gradient)

print("Final w: {}".format(w))


Final w: [-1.73487478e+00  1.99873718e+00  2.84085823e+00  9.98352239e+00
  1.05829633e+00  8.09714200e+00  7.00189057e+00  4.99988469e+00
  8.99983316e+00 -9.50657126e-05  5.99891367e+00]


In [18]:
test_set = test_set.collect()

In [19]:
hit_count = 0
for t_s in test_set:
	if round(1 / (1 + math.exp(-np.dot(w,t_s[0])))) == t_s[1]:
		hit_count +=1

In [22]:
print(f"정확도 : {hit_count/len(test_set)*100}")

정확도 : 49.93571428571428
